In [10]:
from datetime import date
from jugaad_data.nse import bhavcopy_save,bhavcopy_index_save,bhavcopy_fo_save
import pandas as pd
from jugaad_data.holidays import holidays
import sys,os
import mrigutilities as mu

eq_dir = os.path.join('..','data','input')
eq_processed_dir = os.path.join('..','data','processed')


date_range = pd.bdate_range(start='05/16/2024', end = '05/17/2024', 
                         freq='C', holidays = holidays(2024,12))

dates = [x.date() for x in date_range]

print(dates)


[datetime.date(2024, 5, 16), datetime.date(2024, 5, 17)]


In [3]:
from random import randint
import time

for dates in dates:
     try:
        bhavcopy_save(dates, eq_dir)
        bhavcopy_index_save(dates, eq_dir)
        bhavcopy_fo_save(dates, eq_dir)
         
        time.sleep(randint(1,4)) #adding random delay of 1-4 seconds

#      except (ConnectionError, ReadTimeoutError) as e:
     except:
        time.sleep(10) #stop program for 10 seconds and try again.
        try:
            bhavcopy_save(dates, eq_dir)
            bhavcopy_index_save(dates, eq_dir)
            bhavcopy_fo_save(dates, eq_dir)
            time.sleep(randint(1,4))
        except:
            print(f'{dates}: File not Found')

2024-05-16: File not Found
2024-05-17: File not Found


In [11]:
from jugaad_data.nse import NSEArchives
from datetime import datetime, date
import os
import io
import csv
import zipfile
import requests
import pprint
def unzip(function):
    
    def unzipper(*args, **kwargs):
        r = function(*args, **kwargs)
        fp = io.BytesIO(r)
        with zipfile.ZipFile(file=fp) as zf:
            fname = zf.namelist()[0]
            with zf.open(fname) as fp_bh:
                return fp_bh.read().decode('utf-8')
    return unzipper



class NSEArchives_SB(NSEArchives):

    def __init__(self):
        self._routes = {
                "bhavcopy": "/content/historical/EQUITIES/{yyyy}/{MMM}/cm{dd}{MMM}{yyyy}bhav.csv.zip",
                "bhavcopy_full": "/products/content/sec_bhavdata_full_{dd}{mm}{yyyy}.csv",
                "bulk_deals": "/content/equities/bulk.csv",
                "bhavcopy_co": "/content/historical/DERIVATIVES/{yyyy}/{MMM}/co{dd}{MMM}{yyyy}bhav.csv.zip"
            }
        
    def get(self, rout, **params):
        url = self.base_url + self._routes[rout].format(**params)
        print(url)
        self.r = self.s.get(url, timeout=self.timeout)
        return self.r
    @unzip
    def bhavcopy_co_raw(self, dt):
        """Downloads raw bhavcopy text for a specific date"""
        dd = dt.strftime('%d')
        MMM = dt.strftime('%b').upper()
        yyyy = dt.year
        r = self.get("bhavcopy_co", yyyy=yyyy, MMM=MMM, dd=dd)
        return r.content
    
    def bhavcopy_co_save(self, dt, dest, skip_if_present=True):
        """ Saves Derivatives Bhavcopy to a directory """
        fmt = "co%d%b%Ybhav.csv"
        fname = os.path.join(dest, dt.strftime(fmt))
        if os.path.isfile(fname) and skip_if_present:
            return fname
        text = self.bhavcopy_co_raw(dt)
        with open(fname, 'w') as fp:
            fp.write(text)
        return fname

nco = NSEArchives_SB()

from random import randint
import time

for dates in dates:
     try:
        nco.bhavcopy_co_save(dates, eq_dir)
         
        time.sleep(randint(1,4)) #adding random delay of 1-4 seconds

#      except (ConnectionError, ReadTimeoutError) as e:
     except:
        time.sleep(10) #stop program for 10 seconds and try again.
        try:
            nco.bhavcopy_co_save(dates, eq_dir)
            time.sleep(randint(1,4))
        except:
            print(f'{dates}: File not Found')

https://nsearchives.nseindia.com//content/historical/DERIVATIVES/2024/MAY/co16MAY2024bhav.csv.zip
https://nsearchives.nseindia.com//content/historical/DERIVATIVES/2024/MAY/co16MAY2024bhav.csv.zip
2024-05-16: File not Found
https://nsearchives.nseindia.com//content/historical/DERIVATIVES/2024/MAY/co17MAY2024bhav.csv.zip
https://nsearchives.nseindia.com//content/historical/DERIVATIVES/2024/MAY/co17MAY2024bhav.csv.zip
2024-05-17: File not Found


In [66]:
import re,io
import csv
import zipfile

engine = mu.sql_engine()

written_list = []
for r,d,f in os.walk(eq_dir):
    for file in f:
        written = False
        print('Processing File - > ',file)
        if re.search(".*zip",file):
            # print(file)
            zf = zipfile.ZipFile(os.path.join(r,file))
            files = zf.namelist()
            # print(files)
            for f in files:
                # print('FIRST LEVEL   ' ,f)
                # try:
                if re.search(".*zip",f):
                    # print(f)
                    zfiledata = io.BytesIO(zf.read(f))
                    zf1 = zipfile.ZipFile(zfiledata)
                    csvfiles = zf1.namelist()
                    # print(csvfiles)
                    for csvf in csvfiles:
                        # print('SECOND LEVEL    ',str(csvf))
                        if re.search("CO_NSE.*csv",csvf):
                            com_data = pd.read_csv(zf1.open(csvf))
                            dt = datetime.strptime(str(csvf)[9:15],'%d%m%y')
                            com_data.columns = [x.lower() for x in com_data.columns]
                            com_data = com_data[['contract_d','previous_s',	'open_price',	'high_price',	'low_price',	'close_pric',	'settlement','oi_no_con',	'traded_qua']]
                            com_data['value_date'] = dt
                            com_data.to_sql('commodity_fno',engine,index=False,if_exists='append')
                            # print(file, '---> CURRENTLY WRITING 2nd LEVEL ZIP')
                            written = True
                            # print(com_data)
                            # os.replace(os.path.join(r,file), os.path.join(processed_dir, os.path.basename(file)))
                    zfiledata.close()
                    zf1.close()
                # except:
                else:
                    if re.search("CO_NSE.*csv",f):
                        com_data = pd.read_csv(zf.open(f))
                        dt = datetime.strptime(str(f)[9:15],'%d%m%y')
                        com_data.columns = [x.lower() for x in com_data.columns]
                        com_data = com_data[['contract_d','previous_s',	'open_price',	'high_price',	'low_price',	'close_pric',	'settlement','oi_no_con',	'traded_qua']]
                        com_data['value_date'] = dt
                        com_data.to_sql('commodity_fno',engine,index=False,if_exists='append')
                        # print(file, '---> CURRENTLY WRITING 1st LEVEL ZIP' )
                        written= True
        
                        # print(com_data)                 
                        # os.replace(os.path.join(r,file), os.path.join(eq_processed_dir, os.path.basename(file)))
            zf.close()
        if written:
            written_list.append(os.path.join(r,file))
# print(written_list)
for file in written_list:
    # print(file, ' --->  Written ')
    os.replace(file, os.path.join(eq_processed_dir, os.path.basename(file)))
    # print(file,'   ---> MOVED ')

Processing File - >  Axis Mutual Fund_01-Jan-2022_31-Dec-2022.csv
Processing File - >  com_13052024.zip
FIRST LEVEL    CO_NSE_FO130524.csv
com_13052024.zip ---> CURRENTLY WRITING 1st LEVEL ZIP
FIRST LEVEL    CO_NSE_OP130524.csv
com_13052024.zip ---> CURRENTLY WRITING 1st LEVEL ZIP
Processing File - >  com_14052024.zip
FIRST LEVEL    CO_NSE_FO140524.csv
com_14052024.zip ---> CURRENTLY WRITING 1st LEVEL ZIP
FIRST LEVEL    CO_NSE_OP140524.csv
com_14052024.zip ---> CURRENTLY WRITING 1st LEVEL ZIP
Processing File - >  com_15052024.zip
FIRST LEVEL    CO_NSE_FO150524.csv
com_15052024.zip ---> CURRENTLY WRITING 1st LEVEL ZIP
FIRST LEVEL    CO_NSE_OP150524.csv
com_15052024.zip ---> CURRENTLY WRITING 1st LEVEL ZIP
Processing File - >  com_16052024.zip
FIRST LEVEL    CO_NSE_FO160524.csv
com_16052024.zip ---> CURRENTLY WRITING 1st LEVEL ZIP
FIRST LEVEL    CO_NSE_OP160524.csv
com_16052024.zip ---> CURRENTLY WRITING 1st LEVEL ZIP
Processing File - >  com_17052024.zip
FIRST LEVEL    CO_NSE_FO170524.c

In [78]:
import re,io
import csv
import zipfile

engine = mu.sql_engine()
database_name = 'currency_fno'
written_list = []
for r,d,f in os.walk(eq_dir):
    for file in f:
        written = False
        print('Processing File - > ',file)
        if re.search(".*zip",file):
            # print(file)
            zf = zipfile.ZipFile(os.path.join(r,file))
            files = zf.namelist()
            # print(files)
            for f in files:
                # print('FIRST LEVEL   ' ,f)
                # try:
                if re.search(".*zip",f):
                    # print(f)
                    zfiledata = io.BytesIO(zf.read(f))
                    zf1 = zipfile.ZipFile(zfiledata)
                    csvfiles = zf1.namelist()
                    print(csvfiles)
                    for csvf in csvfiles:
                        print('SECOND LEVEL    ',str(csvf))
                        if re.search("CD_NSE.*csv",csvf):
                            com_data = pd.read_csv(zf1.open(csvf))
                            dt = datetime.strptime(str(csvf)[-10:-4],'%d%m%y')
                            com_data.columns = [x.lower() for x in com_data.columns]
                            com_data = com_data[['contract_d','previous_s',	'open_price',	'high_price',	'low_price',	'close_pric',	'settlement','oi_no_con',	'traded_qua']]
                            com_data['value_date'] = dt
                            com_data.to_sql(database_name,engine,index=False,if_exists='append')
                            print(file, '---> CURRENTLY WRITING 2nd LEVEL ZIP')
                            written = True
                            print(com_data)
                            # os.replace(os.path.join(r,file), os.path.join(processed_dir, os.path.basename(file)))
                    zfiledata.close()
                    zf1.close()
                # except:
                else:
                    if re.search("CD_NSE.*csv",f):
                        com_data = pd.read_csv(zf.open(f))
                        dt = datetime.strptime(str(f)[-10:-4],'%d%m%y')
                        com_data.columns = [x.lower() for x in com_data.columns]
                        com_data = com_data[['contract_d','previous_s',	'open_price',	'high_price',	'low_price',	'close_pric',	'settlement','oi_no_con',	'traded_qua']]
                        com_data['value_date'] = dt
                        print(com_data)
                        com_data.to_sql(database_name,engine,index=False,if_exists='append')
                        print(file, '---> CURRENTLY WRITING 1st LEVEL ZIP' )
                        written= True
        
                        print(com_data)                 
                        # os.replace(os.path.join(r,file), os.path.join(eq_processed_dir, os.path.basename(file)))
            zf.close()
        if written:
            written_list.append(os.path.join(r,file))
# print(written_list)
for file in written_list:
    # print(file, ' --->  Written ')
    os.replace(file, os.path.join(eq_processed_dir, os.path.basename(file)))
    # print(file,'   ---> MOVED ')

Processing File - >  Axis Mutual Fund_01-Jan-2022_31-Dec-2022.csv
Processing File - >  Reports-Archives-Multiple-01032024 (1).zip
['CD_NSE_FO010324.csv', 'CD_NSE_OP010324.csv']
SECOND LEVEL     CD_NSE_FO010324.csv
Reports-Archives-Multiple-01032024 (1).zip ---> CURRENTLY WRITING 2nd LEVEL ZIP
                  contract_d  previous_s  open_price  high_price  low_price  \
0    FUTCURUSDINR15-MAR-2024       82.96     82.9000      82.940    82.8850   
1    FUTCURGBPINR26-JUN-2024      105.32    105.0775     105.150   105.0075   
2    FUTCURUSDINR28-AUG-2024       83.49     83.4625      83.480    83.4300   
3    FUTCURUSDINR27-DEC-2024       84.02     84.0200      84.020    83.9400   
4    FUTCURGBPINR26-MAR-2024      105.08    104.8775     104.880   104.7200   
..                       ...         ...         ...         ...        ...   
127  FUTCUREURINR10-MAY-2024       90.26      0.0000       0.000     0.0000   
128  FUTCURJPYINR17-MAY-2024       56.03      0.0000       0.000     0.000

In [4]:
import requests
from io import StringIO

urls = {'NIFTY 100' : 'https://nsearchives.nseindia.com/content/indices/ind_nifty100list.csv',
        'NIFTY 50' : 'https://nsearchives.nseindia.com/content/indices/ind_nifty50list.csv'
       }
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0'}

result = {}
# print(headers)
for key in urls.keys():
    req = requests.get(urls[key], headers=headers)
    # print(req)
    data = StringIO(req.text)
    df = pd.read_csv(data)
    result[key] = str(df['Symbol'].tolist())
engine = mu.sql_engine()
date = engine.execute("select max(date) from stock_history where symbol='NIFTY 50'").fetchall()[0][0]
disable_sql = "alter table stock_history disable trigger return_trigger"
enable_sql = "alter table stock_history enable trigger return_trigger"
engine.execute(disable_sql)
print(str(date).replace('-',''))
print("Populating Index Constituent")
# usql =  "update stock_history set index_members = %s where symbol = %s and date = %s"
# engine.execute(usql,(result['NIFTY 50'],'NIFTY 50',(str(date).replace('-',''))))
for key in result.keys():
    usql = ""
    usql =  "update stock_history set index_members = %s where symbol = %s and date = %s"
    engine.execute(usql,(result[key],key,(str(date).replace('-',''))))
# n50 = pd.read_csv(n50_url)
# n100 = pd.read_csv(n100_url)

# print(n50)

20240403
Populating Index Constituent


In [20]:
import json
nifty_100 = engine.execute("select index_members from stock_history where symbol='NIFTY 100' and index_members is not NULL order by date desc limit 1").fetchall()[0][0]
nifty_100 = nifty_100.strip('][').split(', ')
nifty_100 = [x[1:-1] for x in nifty_100]
nifty_100

['ABB',
 'ADANIENSOL',
 'ADANIENT',
 'ADANIGREEN',
 'ADANIPORTS',
 'ADANIPOWER',
 'ATGL',
 'AMBUJACEM',
 'APOLLOHOSP',
 'ASIANPAINT',
 'DMART',
 'AXISBANK',
 'BAJAJ-AUTO',
 'BAJFINANCE',
 'BAJAJFINSV',
 'BAJAJHLDNG',
 'BANKBARODA',
 'BERGEPAINT',
 'BEL',
 'BPCL',
 'BHARTIARTL',
 'BOSCHLTD',
 'BRITANNIA',
 'CANBK',
 'CHOLAFIN',
 'CIPLA',
 'COALINDIA',
 'COLPAL',
 'DLF',
 'DABUR',
 'DIVISLAB',
 'DRREDDY',
 'EICHERMOT',
 'GAIL',
 'GODREJCP',
 'GRASIM',
 'HCLTECH',
 'HDFCBANK',
 'HDFCLIFE',
 'HAVELLS',
 'HEROMOTOCO',
 'HINDALCO',
 'HAL',
 'HINDUNILVR',
 'ICICIBANK',
 'ICICIGI',
 'ICICIPRULI',
 'ITC',
 'IOC',
 'IRCTC',
 'IRFC',
 'INDUSINDBK',
 'NAUKRI',
 'INFY',
 'INDIGO',
 'JSWSTEEL',
 'JINDALSTEL',
 'JIOFIN',
 'KOTAKBANK',
 'LTIM',
 'LT',
 'LICI',
 'M&M',
 'MARICO',
 'MARUTI',
 'NTPC',
 'NESTLEIND',
 'ONGC',
 'PIDILITIND',
 'PFC',
 'POWERGRID',
 'PNB',
 'RECLTD',
 'RELIANCE',
 'SBICARD',
 'SBILIFE',
 'SRF',
 'MOTHERSON',
 'SHREECEM',
 'SHRIRAMFIN',
 'SIEMENS',
 'SBIN',
 'SUNPHARMA',
 'TVS

In [76]:
a = 'CD_NSE_FO010324.csv'
a[-10:-4]

'010324'